In [578]:
import pandas as pd
import re
import os

In [580]:
def clean_description(s: str) -> list[str]:
    """
    Функция очистки принимает на вход строку-описание, в которой могут быть лишние символы (пуктуации, скобки и т.д.) 
    и буквы в разном регистре. Убирает лишние символы (кроме точек, точки используют для сокращения слов и их можно 
    принять как разделители между словами), приводит все буквы к меньшему регистру, добавляет пробел (между цифрами/буквами 
    ('Соль100гр' -> 'соль 100 гр')) и выводит список из слов.
    """
    for ch in '[@")(*&+\,.|/;!?%^-_]':
        s = s.replace(ch, '')
    s = s.replace("''", '').lower()
    s1 = ''
    for word in s.split():
        i = 0
        while i < len(word)-1:
            s1 += word[i]
            if (word[i].isdigit() and word[i+1].isalpha()) or (word[i].isalpha() and word[i+1].isdigit()):
                s1 += ' '
                i += 1
            else:
                i += 1
        s1 += word[i] + ';'
    s1 = s1[:-1]
    return re.split('[ ;]', s1)

In [581]:
def func_affiliation(df: pd.DataFrame) -> pd.DataFrame:
    """
    Функция для каждой группы карточек создает счетчик-словарь слов из описаний товаров. 
    При этом отдается предпочтение первым словам в описании товаров, с длиной не менее 2.
    В датафрейме создаем временную колонку d_total_for_cat со значениями мощности словарей по каждой категории (карточке)
    В колонку 'результат' для каждого описания товара указываем процент соотвествия товара этой категории (карточке).
    """
    # Применяем функцию очистки для каждой записи поля "реал. описание баркода"
    df.iloc[:, 2] = df.iloc[:, 2].apply(lambda x: clean_description(x))
    n = 0
    list_d_total = []
    for i in df.iloc[:, 0].unique():  # Проходимся по уникальным номерам карточек
        df_currently = df[df.iloc[:, 0] == i]  # Рассматириваем только часть исходного фрейма, где значения карточек идентичны
        d = {}  # Создаем счетчик значений слов в категории
        for desc in range(len(df_currently)):  # Проходим по каждой строке локального фрейма и формируем локальный счетчик-словарь
            one_entry = df_currently.iloc[desc,2]  # Сохраняем одну строку в переменную для более удобного чтения кода далее
            for i in range(len(one_entry)):       
                if not one_entry[i].isdigit() and len(one_entry[i]) >= 2 and i < 2:
                    d[one_entry[i]] = d.get(one_entry[i],0) + 10  # Отдаем предпочтение первым словам, состоящих не из цифр
                else:
                    d[one_entry[i]] = d.get(one_entry[i],0) + 1
        for desc in range(len(df_currently)):
            df.loc[n, 'результат'] = sum(d[word] for word in df_currently.iloc[desc, 2])
            n += 1
        for _ in range(len(df_currently)):
            list_d_total.append(sum(d.values()))
    df['d_total_for_cat'] = list_d_total
    df['результат'] = ((df['результат'] / df['d_total_for_cat']) * 100).round(2).astype(str) + '%'
    return df

In [582]:
path_input = './Data_Excel_Input'
extension = '.xlsx'

all_files = [file for file in os.listdir(path_input) if file.endswith('.xlsx')]

for file in all_files:
    df_init = pd.read_excel(os.path.join(path, file))
    df_init = df_init[df_init.iloc[:, 2].notnull()]  # Убираем из рассмотрения те записи, где остуствтует описание товара
    df_init = df_init.reset_index(drop=True)
    df = df_init.copy()  # Копируем исходный фрейм во временный, в котором будем делать изменения в поле описания товара
    func_affiliation(df)
    path_output = './Data_Excel_Output/' + 'Results_of_' + file + '.xlsx'
    df = pd.concat([df_init.iloc[:,:3], df.iloc[:,3]], axis=1)
    df.to_excel(path_output)